In [39]:
from cnn_image_generation import *
from analysis_nn import *
from update_library import *
import random

from PIL import Image, ImageFilter
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

lib = update_library()

Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with bounding box
Please update object with

In [374]:
num_pics = 925
# Sampling bounds (x,y)
sample_dim = 2
images_domain = [[52,72],[1,19]]
modification_domain = [[0,1],[0,1]]

path_synth_set = './synth/uniform/'

In [375]:
def scale_sample(sample, sampling_domain):
    '''Scale a [0,1] sample in a given domain'''
    for i in range(len(sample)):
        sample[i] = sample[i]*(sampling_domain[i][1] - sampling_domain[i][0]) + sampling_domain[i][0]
    return sample    

def random_sample(typ,sampling_domain):
    sample = []
    for i in range(len(sampling_domain)):
        domain = sampling_domain[i]
        if typ == 'float':
            r = random.random()*(domain[1]-domain[0])+domain[0]
        elif typ == 'int':
            r = random.choice(range(domain[0],domain[1]))
        else:
            print('Error')
        sample.append(r)
    return sample

def box_2_kitti_format(box):
    '''Transform box for KITTI label format'''
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    left = int(x - w/2)
    right = int(x + w/2)
    top = int(y - h/2)
    bot = int(y + h/2)
    return [left,top,right,bot]
    

In [ ]:
for i in range(num_pics):
    if (i % 25 == 0):
        print(i)
    
    sample = random_sample('int',images_domain) + random_sample('float',modification_domain)
    img, box = gen_comp_img(lib, bg_id=sample[0], fg_id=sample[1],x=sample[2], y=sample[3])

    file_name = 's' + str(i).zfill(6)
    # Save image
    img.save(path_synth_set + 'images/' + file_name + '.png')

    # Save label
    f = open(path_synth_set + 'labels/' + file_name + '.txt', 'w')
    label = [0,0,0] + box_2_kitti_format(box) + [0,0,0,0,0,0,0]
    label = list(map(str, label))
    label = " ".join(label)
    label  = "Car " + label
    f.write(label)  # python will convert \n to os.linesep
    f.close()

0
25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775
800
825
850


In [144]:
img, box = gen_comp_img(lib, bg_id=69, fg_id=19,x=sample[2], y=sample[3])

IndexError: list index out of range

In [15]:
scale_sample([0.5,0.7], [[0,1],[0,0.5]])

[0.5, 0.35]

In [18]:
random.random(2)

TypeError: random() takes no arguments (1 given)

In [22]:
 random_sample(5)

[0.8365261050663598,
 0.6500198560236171,
 0.5290232094944505,
 0.9244035357725716,
 0.3036994618940815]

2

In [365]:
list1 = str([1,2,3])
" ".join(list1)

'[ 1 ,   2 ,   3 ]'

In [357]:
str1

'1 2 3'

'1 2 3'